# Time-series Available Bike Forecasting

## Introduction

As part of the COMP30830 module there is a requirement to develop a basic predictive model for the number of available bikes/stands dependant on weather.

#### Automated Process

In [ ]:

####--------------------------------------
#00.Import Modules
####--------------------------------------

######---------BEGIN
#      ML
######--------END

import nltk as nl
import sklearn as sk
import matplotlib as mp
import xgboost as xg
import pymc3 as pymc
import sympy as sym

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score



######---------BEGIN
#      SQL
######--------END


import requests as rq
import sqlalchemy as sqla
#import pyodbc
#import cx_oracle as cx


######---------BEGIN
#     GENERAL
######--------END

import pandas as pd
import datetime as dt
import numpy as np
import sys
import os
import json
import time
import socket
import traceback as tb
import platform
import json
import pprint
import pickle

######---------BEGIN
#     DATA VIS
######--------END

import seaborn as sns
import matplotlib as mp
#from bokeh import *
#from dash import *

import matplotlib.pyplot as plt

In [ ]:
##------------------APP---------------##
## 
##User:     Disappster
##DC:       2021-03-24
##DLM:      2021-03-24
##MC:       COMP30830
##SD:       SQL Statements
##LD:       This section should hold all of the CODEBASE for SQL commands. SQL Commands should not exist in other app areas.
##
##------------------APP---------------##


####
#---01. Station Queries
####

####All Stations
SQL_select_station="""
    SELECT
         stat.{}                              AS                      number
        ,stat.{}                              AS                      address
        ,stat.{}                              AS                      banking
        ,stat.{}                              AS                      bike_status
        ,stat.{}                              AS                      bike_stands
        ,stat.{}                              AS                      contract_name
        ,stat.{}                              AS                      name
        ,stat.{}                              AS                      position_lat
        ,stat.{}                              AS                      position_long
        ,FROM_UNIXTIME(stat.{})               AS                      created_date
    FROM
        {} stat
    """.format('number'
            ,'address'
            ,'banking'
            ,'bike_stands'
            ,'bonus'
            ,'contract_name'
            ,'name'
            ,'position_lat'
            ,'position_long'
            ,'created_date'
            ,'01_station')



###Stations Given Station Number
SQL_select_station_where_number="""
    SELECT
         stat.{}                              AS                      number
        ,stat.{}                              AS                      address
        ,stat.{}                              AS                      banking
        ,stat.{}                              AS                      bike_status
        ,stat.{}                              AS                      bike_stands
        ,stat.{}                              AS                      contract_name
        ,stat.{}                              AS                      name
        ,stat.{}                              AS                      position_lat
        ,stat.{}                              AS                      position_long
        ,FROM_UNIXTIME(stat.{})               AS                      created_date
    FROM
        {} stat
    WHERE
        stat.{}
    """.format('number'
                ,'address'
                ,'banking'
                ,'bike_stands'
                ,'bonus'
                ,'contract_name'
                ,'name'
                ,'position_lat'
                ,'position_long'
                ,'created_date'
                ,'01_station'
                ,'number') + """={}"""


####
#---02. Availability Queries
####

SQL_select_availability="""
    SELECT
         avail.{}                               AS              number
        ,avail.{}                               AS              available_bikes
        ,avail.{}                               AS              available_bike_stands
        ,FROM_UNIXTIME({})                      AS              last_update
        ,FROM_UNIXTIME({})                      AS              created_date
    FROM
        {} avail
    """.format(
            'number'
            ,'available_bikes'
            ,'available_bike_stands'
            ,'last_update'
            ,'created_date'
            ,'01_availability'
            )


SQL_select_availability_where_number="""
    SELECT
         avail.{}                            AS              number
        ,avail.{}                            AS              available_bikes
        ,avail.{}                            AS              available_bike_stands
        ,FROM_UNIXTIME(avail.{})             AS              last_update
        ,FROM_UNIXTIME(avail.{})             AS              created_date
    FROM
        {} avail
    WHERE
        avail.{}""".format(
            'number'
            ,'available_bikes'
            ,'available_bike_stands'
            ,'last_update'
            ,'created_date'
            ,'01_availability'
            ,'number'
            ) + "={}"

#Get the Last Update Availability Info for each station
SQL_select_availability_last_update="""
    SELECT
         all_avail.{}                            AS              number
        ,all_avail.{}                            AS              available_bikes
        ,all_avail.{}                            AS              available_bike_stands
        ,FROM_UNIXTIME(all_avail.{})             AS              last_update
        ,FROM_UNIXTIME(all_avail.{})             AS              created_date
    FROM
        (    
            SELECT
                 {}                  AS              number
                ,MAX({})             AS              created_date
            FROM
                {}
            GROUP BY
                {}
        ) max_avail
        
    INNER JOIN
        {} all_avail
    ON
        all_avail.{}=max_avail.number
    AND
        all_avail.{}=max_avail.created_date

    """.format('number'
                ,'available_bikes'
                ,'available_bike_stands'
                ,'last_update'
                ,'created_date'

                ,'number'
                ,'created_date'
                ,'01_availability'
                ,'number'

                ,'01_availability'
                ,'number'
                ,'created_date'
            )

####
#---03. Weather Queries
####

SQL_select_weather="""
    SELECT
         weath.{}                             AS                        number
        ,weath.{}                             AS                        position_long
        ,weath.{}                             AS                        position_lat
        ,weath.{}                             AS                        weather_id
        ,weath.{}                             AS                        main
        ,weath.{}                             AS                        description
        ,weath.{}                             AS                        icon
        ,weath.{}                             AS                        icon_url
        ,weath.{}                             AS                        base
        ,weath.{}                             AS                        temp
        ,weath.{}                             AS                        feels_like
        ,weath.{}                             AS                        temp_min
        ,weath.{}                             AS                        temp_max
        ,weath.{}                             AS                        pressure
        ,weath.{}                             AS                        humidity
        ,weath.{}                             AS                        visibility
        ,weath.{}                             AS                        wind_speed
        ,weath.{}                             AS                        wind_degree
        ,weath.{}                             AS                        clouds_all
        ,weath.{}                             AS                        datetime
        ,weath.{}                             AS                        sys_id
        ,weath.{}                             AS                        sys_country
        ,weath.{}                             AS                        sys_sunrise
        ,weath.{}                             AS                        sys_sunset
        ,weath.{}                             AS                        sys_type
        ,weath.{}                             AS                        timezone
        ,weath.{}                             AS                        id
        ,weath.{}                             AS                        weather_name
        ,weath.{}                             AS                        cod
        ,FROM_UNIXTIME(weath.{})              AS                        created_date
    FROM
        {} weath
    """.format('number'
                ,'position_long'
                ,'position_lat'
                ,'weather_id'
                ,'main'
                ,'description'
                ,'icon'
                ,'icon_url'
                ,'base'
                ,'temp'
                ,'feels_like'
                ,'temp_min'
                ,'temp_max'
                ,'pressure'
                ,'humidity'
                ,'visibility'
                ,'wind_speed'
                ,'wind_degree'
                ,'clouds_all'
                ,'datetime'
                ,'sys_id'
                ,'sys_country'
                ,'sys_sunrise'
                ,'sys_sunset'
                ,'sys_type'
                ,'timezone'
                ,'id'
                ,'name'
                ,'cod'
                ,'created_date'
                ,'01_weather')


SQL_select_weather_where_number="""
    SELECT
         weath.{}                             AS                        number
        ,weath.{}                             AS                        position_long
        ,weath.{}                             AS                        position_lat
        ,weath.{}                             AS                        weather_id
        ,weath.{}                             AS                        main
        ,weath.{}                             AS                        description
        ,weath.{}                             AS                        icon
        ,weath.{}                             AS                        icon_url
        ,weath.{}                             AS                        base
        ,weath.{}                             AS                        temp
        ,weath.{}                             AS                        feels_like
        ,weath.{}                             AS                        temp_min
        ,weath.{}                             AS                        temp_max
        ,weath.{}                             AS                        pressure
        ,weath.{}                             AS                        humidity
        ,weath.{}                             AS                        visibility
        ,weath.{}                             AS                        wind_speed
        ,weath.{}                             AS                        wind_degree
        ,weath.{}                             AS                        clouds_all
        ,weath.{}                             AS                        datetime
        ,weath.{}                             AS                        sys_id
        ,weath.{}                             AS                        sys_country
        ,weath.{}                             AS                        sys_sunrise
        ,weath.{}                             AS                        sys_sunset
        ,weath.{}                             AS                        sys_type
        ,weath.{}                             AS                        timezone
        ,weath.{}                             AS                        id
        ,weath.{}                             AS                        weather_name
        ,weath.{}                             AS                        cod
        ,FROM_UNIXTIME(weath.{})              AS                        created_date
    FROM
        {} weath
    WHERE
        weath.{}
    """.format('number'
                ,'position_long'
                ,'position_lat'
                ,'weather_id'
                ,'main'
                ,'description'
                ,'icon'
                ,'icon_url'
                ,'base'
                ,'temp'
                ,'feels_like'
                ,'temp_min'
                ,'temp_max'
                ,'pressure'
                ,'humidity'
                ,'visibility'
                ,'wind_speed'
                ,'wind_degree'
                ,'clouds_all'
                ,'datetime'
                ,'sys_id'
                ,'sys_country'
                ,'sys_sunrise'
                ,'sys_sunset'
                ,'sys_type'
                ,'timezone'
                ,'id'
                ,'name'
                ,'cod'
                ,'created_date'
                ,'01_weather'
                ,'number')+"={}"



SQL_select_weather_last_update="""
    SELECT
         weath.{}                             AS                        number
        ,weath.{}                             AS                        position_long
        ,weath.{}                             AS                        position_lat
        ,weath.{}                             AS                        weather_id
        ,weath.{}                             AS                        main
        ,weath.{}                             AS                        description
        ,weath.{}                             AS                        icon
        ,weath.{}                             AS                        icon_url
        ,weath.{}                             AS                        base
        ,weath.{}                             AS                        temp
        ,weath.{}                             AS                        feels_like
        ,weath.{}                             AS                        temp_min
        ,weath.{}                             AS                        temp_max
        ,weath.{}                             AS                        pressure
        ,weath.{}                             AS                        humidity
        ,weath.{}                             AS                        visibility
        ,weath.{}                             AS                        wind_speed
        ,weath.{}                             AS                        wind_degree
        ,weath.{}                             AS                        clouds_all
        ,weath.{}                             AS                        datetime
        ,weath.{}                             AS                        sys_id
        ,weath.{}                             AS                        sys_country
        ,weath.{}                             AS                        sys_sunrise
        ,weath.{}                             AS                        sys_sunset
        ,weath.{}                             AS                        sys_type
        ,weath.{}                             AS                        timezone
        ,weath.{}                             AS                        id
        ,weath.{}                             AS                        weather_name
        ,weath.{}                             AS                        cod
        ,FROM_UNIXTIME(weath.{})              AS                        created_date
    FROM
        (SELECT
            {}                                AS                        number
            ,MAX({})                          AS                        created_date
        FROM
            {}
        GROUP BY
            {}
        ) max_weath

    INNER JOIN
        {} weath
    ON
        max_weath.created_date=weath.{}
        AND
        max_weath.number=weath.{}
    """.format('number'
                ,'position_long'
                ,'position_lat'
                ,'weather_id'
                ,'main'
                ,'description'
                ,'icon'
                ,'icon_url'
                ,'base'
                ,'temp'
                ,'feels_like'
                ,'temp_min'
                ,'temp_max'
                ,'pressure'
                ,'humidity'
                ,'visibility'
                ,'wind_speed'
                ,'wind_degree'
                ,'clouds_all'
                ,'datetime'
                ,'sys_id'
                ,'sys_country'
                ,'sys_sunrise'
                ,'sys_sunset'
                ,'sys_type'
                ,'timezone'
                ,'id'
                ,'name'
                ,'cod'
                ,'created_date'

                ,'number'
                ,'created_date'
                ,'01_weather'
                ,'number'

                ,'01_weather'
                ,'created_date'
                ,'number'
                )



####
#---04. Joining Queries
####


SQL_select_avail_weather="""
    SELECT
        available.number                              AS          number
        ,available.available_bikes                    AS          available_bikes
        ,available.available_bike_stands              AS          available_bike_stands
        ,available.last_update                        AS          last_update
        ,available.created_date                       AS          created_date
        ,weather.position_long                        AS          weather_position_long
        ,weather.position_lat                         AS          weather_position_lat
        ,weather.weather_id
        ,weather.main
        ,weather.description
        ,weather.icon
        ,weather.icon_url
        ,weather.base
        ,weather.temp
        ,weather.feels_like
        ,weather.temp_min
        ,weather.temp_max
        ,weather.pressure
        ,weather.humidity
        ,weather.visibility
        ,weather.wind_speed
        ,weather.wind_degree
        ,weather.clouds_all
        ,weather.datetime
        ,weather.sys_id
        ,weather.sys_country
        ,weather.sys_sunrise
        ,weather.sys_sunset
        ,weather.sys_type
        ,weather.timezone
        ,weather.id
        ,weather.name                                   AS              weather_name
        ,weather.cod
    FROM
        ({}) available

    INNER JOIN
        ({}) weather
    ON
        available.number=weather.number
        AND
        available.created_date=weather.created_date
    """.format(SQL_select_availability
           ,SQL_select_weather)




SQL_select_station_avail="""
    SELECT
        station.number                                AS                      number
        ,station.address                              AS                      address
        ,station.banking                              AS                      banking
        ,station.bike_status                          AS                      bike_status
        ,station.bike_stands                          AS                      bike_stands
        ,station.contract_name                        AS                      contract_name
        ,station.name                                 AS                      name
        ,station.position_lat                         AS                      position_lat
        ,station.position_long                        AS                      position_long
        ,available.available_bikes                    AS                      available_bikes
        ,available.available_bike_stands              AS                      available_bike_stands
        ,available.last_update                        AS                      last_update
        ,available.created_date                       AS                      created_date
    FROM
        ({}) station

    INNER JOIN
        ({}) available
    ON
        station.number=available.number
    """.format(SQL_select_station
           ,SQL_select_availability)


SQL_select_station_avail_conditionals="""
    SELECT
        station.number                                AS                      number
        ,station.address                              AS                      address
        ,station.banking                              AS                      banking
        ,station.bike_status                          AS                      bike_status
        ,station.bike_stands                          AS                      bike_stands
        ,station.contract_name                        AS                      contract_name
        ,station.name                                 AS                      name
        ,station.position_lat                         AS                      position_lat
        ,station.position_long                        AS                      position_long
        ,available.available_bikes                    AS                      available_bikes
        ,available.available_bike_stands              AS                      available_bike_stands
        ,available.last_update                        AS                      last_update
        ,available.created_date                       AS                      created_date
    FROM
        ({}) station

    INNER JOIN
        ({}) available
    ON
        station.number=available.number
    """

SQL_select_station_avail_latest_update="""
    SELECT
        station.number                                AS                      number
        ,station.address                              AS                      address
        ,station.banking                              AS                      banking
        ,station.bike_status                          AS                      bike_status
        ,station.bike_stands                          AS                      bike_stands
        ,station.contract_name                        AS                      contract_name
        ,station.name                                 AS                      name
        ,station.position_lat                         AS                      position_lat
        ,station.position_long                        AS                      position_long
        ,available.available_bikes                    AS                      available_bikes
        ,available.available_bike_stands              AS                      available_bike_stands
        ,available.last_update                        AS                      last_update
        ,available.created_date                       AS                      created_date
    FROM
        ({}) station

    INNER JOIN
        ({}) available
    ON
        station.number=available.number
    """.format(SQL_select_station,SQL_select_availability_last_update)


SQL_select_station_avail_weather="""
    SELECT
        station.number                                AS                      number
        ,station.address                              AS                      address
        ,station.banking                              AS                      banking
        ,station.bike_status                          AS                      bike_status
        ,station.bike_stands                          AS                      bike_stands
        ,station.contract_name                        AS                      contract_name
        ,station.name                                 AS                      name
        ,station.position_lat                         AS                      position_lat
        ,station.position_long                        AS                      position_long
        ,available.available_bikes                    AS                      available_bikes
        ,available.available_bike_stands              AS                      available_bike_stands
        ,available.last_update                        AS                      last_update
        ,available.created_date                       AS                      created_date
        ,weather.position_long                        AS                      weather_position_long
        ,weather.position_lat                         AS                      weather_position_lat
        ,weather.weather_id
        ,weather.main
        ,weather.description
        ,weather.icon
        ,weather.icon_url
        ,weather.base
        ,weather.temp
        ,weather.feels_like
        ,weather.temp_min
        ,weather.temp_max
        ,weather.pressure
        ,weather.humidity
        ,weather.visibility
        ,weather.wind_speed
        ,weather.wind_degree
        ,weather.clouds_all
        ,weather.datetime
        ,weather.sys_id
        ,weather.sys_country
        ,weather.sys_sunrise
        ,weather.sys_sunset
        ,weather.sys_type
        ,weather.timezone
        ,weather.id
        ,weather.weather_name
        ,weather.cod
    FROM
        ({}) station

    INNER JOIN
        ({}) available
    ON
        station.number=available.number

    INNER JOIN
        ({}) weather
    ON
        available.number=weather.number
        AND
        available.created_date=weather.created_date
    """.format(SQL_select_station
           ,SQL_select_availability
           ,SQL_select_weather)




SQL_select_avail_weather_conditional="""
    SELECT
        available.number                              AS          number
        ,available.available_bikes                    AS          available_bikes
        ,available.available_bike_stands              AS          available_bike_stands
        ,available.last_update                        AS          last_update
        ,available.created_date                       AS          created_date
        ,weather.position_long                        AS          weather_position_long
        ,weather.position_lat                         AS          weather_position_lat
        ,weather.weather_id
        ,weather.main
        ,weather.description
        ,weather.icon
        ,weather.icon_url
        ,weather.base
        ,weather.temp
        ,weather.feels_like
        ,weather.temp_min
        ,weather.temp_max
        ,weather.pressure
        ,weather.humidity
        ,weather.visibility
        ,weather.wind_speed
        ,weather.wind_degree
        ,weather.clouds_all
        ,weather.datetime
        ,weather.sys_id
        ,weather.sys_country
        ,weather.sys_sunrise
        ,weather.sys_sunset
        ,weather.sys_type
        ,weather.timezone
        ,weather.id
        ,weather.weather_name
        ,weather.cod
    FROM
        ({}) available

    INNER JOIN
        ({}) weather
    ON
        available.number=weather.number
        AND
        available.created_date=weather.created_date
    """

SQL_select_station_avail_weather_conditionals="""
    SELECT
        station.number                                AS                      number
        ,station.address                              AS                      address
        ,station.banking                              AS                      banking
        ,station.bike_status                          AS                      bike_status
        ,station.bike_stands                          AS                      bike_stands
        ,station.contract_name                        AS                      contract_name
        ,station.name                                 AS                      name
        ,station.position_lat                         AS                      position_lat
        ,station.position_long                        AS                      position_long
        ,available.available_bikes                    AS                      available_bikes
        ,available.available_bike_stands              AS                      available_bike_stands
        ,available.last_update                        AS                      last_update
        ,available.created_date                       AS                      created_date
        ,weather.position_long                        AS                      weather_position_long
        ,weather.position_lat                         AS                      weather_position_lat
        ,weather.weather_id
        ,weather.main
        ,weather.description
        ,weather.icon
        ,weather.icon_url
        ,weather.base
        ,weather.temp
        ,weather.feels_like
        ,weather.temp_min
        ,weather.temp_max
        ,weather.pressure
        ,weather.humidity
        ,weather.visibility
        ,weather.wind_speed
        ,weather.wind_degree
        ,weather.clouds_all
        ,weather.datetime
        ,weather.sys_id
        ,weather.sys_country
        ,weather.sys_sunrise
        ,weather.sys_sunset
        ,weather.sys_type
        ,weather.timezone
        ,weather.id
        ,weather.weather_name
        ,weather.cod
    FROM
        ({}) station

    INNER JOIN
        ({}) available
    ON
        station.number=available.number

    INNER JOIN
        ({}) weather
    ON
        available.number=weather.number
        AND
        available.created_date=weather.created_date
    """



SQL_select_limit_availability= """SELECT 
                                        * 
                                FROM 
                                    {}
                                ORDER BY 
                                    created_date 
                                DESC
                                    LIMIT 109;""".format(
                                            '01_availability'
                                    )

In [ ]:


database_dictionary={
                        'username':'adamryan'
                        ,'password':'adam.ryan1'
                        ,'database':'dbbikes'
                        ,'endpoint':'dbbikes.cmbuuvrlonfv.us-east-1.rds.amazonaws.com'
                        ,'port':'3306'    
                    }

In [ ]:

def connect_db_engine(host,user,password,port,db):
    """Connect to the db engine
    
    host: host
    user: user
    password: pw
    port: port
    db: Name of DB
    
    
    """
    
    print("Inside connect_db_engine()\n\n")
    
    error_code=0
    engine=''
    
    error_dictionary={0:'No Error'
                     ,1:'One of the parameters is wrong'
                      ,999: 'Uncaught exception'
                     }
    
    try:
        connect_statement='mysql+mysqlconnector://{}:{}@{}:{}/{}'.format(user,password,host,port,db)
        print(connect_statement)
        engine=sqla.create_engine(connect_statement,echo=True)
        
    except Exception as e:
        error_code=999
        print(e)
    
    return [error_code,engine]





In [ ]:

def station_availability_weather_table_df(host,user,password,port,db):
    """This function pulls the station, weather, availability data.
    
    Note: This is very time intensive. Use this to pass to other summary functions"""
    
    print("Inside pull_station_weather_availability_data(host,user,password,port,db)")
    
    #Possible Errors
    error_dictionary={0:'No Error'
                     ,1:'The database failed to connect'
                     ,2:"The query is not a valid string"
                     ,3: "The returned database is empty"
                      ,999: 'Uncaught exception'
                     }
    
    #Set up a default value to return
    data_df=pd.DataFrame()
    
    error_code=0
    
    #Configure the SQL statement
    sql_statement=SQL_select_station_avail_weather
    
    time_statement="The retrieval from the database took: {} (ns)"
    
    #Begin try
    try:
        engine_l=connect_db_engine(host,user,password,port,db)
        engine=engine_l[1]
        
        #No error connecting to engine
        if engine_l[0]==0:
            
            #String
            if type(sql_statement)==str and len(sql_statement)>0:
                
                #Begin counter
                start_time=time.perf_counter_ns()
                data_df=pd.read_sql(sql_statement,engine)
                end_time=time.perf_counter_ns()
                engine.dispose()
                
                #Performance measurement
                print(time_statement.format(end_time-start_time))
                
                #Dataframe is empty
                if len(data_df)==0:
                    error_code=3
                    error_message=error_dictionary[error_code]
                    
            #Invalid SQL Statement
            else:
                error_code=2
                error_message=error_dictionary[error_code]         
        
        else:
            error_code=1
            error_message=error_dictionary[error_code]

    except Exception as e:
        error_code=999
        print("Unexpected failure: {}".format(e))
        
    return data_df




In [ ]:

def add_time_features(df, date_time_column):
    """
    Various time features for analytics
    """
    print(df)
    #Features to keep
    #df['timestamp']=(df[date_time_column].astype(int)/10**9).astype(int)
    df['hour'] = df[date_time_column].dt.hour
    df['dayofweek'] = df[date_time_column].dt.dayofweek
    df['dayofmonth'] = df[date_time_column].dt.day
    df['dayofyear'] = df[date_time_column].dt.dayofyear
    
    df['bool_weekend']=np.where(df['dayofweek']>4, True, False)
    #df['bool_level5']=np.where(df['dayofyear']<pd.to_datetime('2021-05-04').dt.day, True, False)
    
    #Bank Holidays and weekend
    df['bool_dayoff']=False
    
    df.loc[(df['dayofweek']>4) |
       (df['dayofyear']==pd.to_datetime('2021-03-17').dayofyear) | 
       (df['dayofyear']==pd.to_datetime('2021-04-05').dayofyear) | 
        (df['dayofyear']==pd.to_datetime('2021-05-04').dayofyear),
       'bool_dayoff'] = True
    
    #Work Hours 9am to 5pm
    df['bool_workhour']=True
    df.loc[(df['hour']>8) &
            (df['hour']<16),'bool_workhour']=False
    
    df.loc[(df['dayofweek']>4) |
       ((df['dayofyear']==pd.to_datetime('2021-03-17').dayofyear) |
       (df['dayofyear']==pd.to_datetime('2021-04-05').dayofyear) |
        (df['dayofyear']==pd.to_datetime('2021-05-04').dayofyear)),
       'bool_dayoff'] = True   
    
    df['bool_commutehour']=False
    df.loc[((df['hour']>=7) & (df['hour']<=8)) |
           ((df['hour']>=16) & (df['hour']<=17)),'bool_commutehour']= True
       
    #9pm - 5am
    df['bool_night']=False
    df.loc[(df['hour']>20) | 
            (df['hour']<6),'bool_night']=True
                        
    #Poor Results:
    #df['quarter'] = df[date_time_column].dt.quarter
    df['month'] = df[date_time_column].dt.month
    df['year'] = df[date_time_column].dt.year
    df['minute'] = df[date_time_column].dt.minute
    #df['weekofyear'] = df[date_time_column].dt.weekofyear
    #df=d#f.drop(date_time_column, axis=1)

    df=df.drop('dayofyear',axis=1)
    
    return [df,['minute','hour','dayofweek','dayofmonth','month','year','bool_weekend','bool_dayoff','bool_workhour','bool_commutehour','bool_night']]

In [ ]:
def get_test_date(df,time_column,test_set_size=0.2, verbose=False):
    """A function to determine what date should be our test date"""
    
    #Default Date
    datetime_at_test_limit=pd.to_datetime('2020-04-01')

    sorted_datetimes=df[time_column].sort_values().unique()

    number_of_datetimes=len(sorted_datetimes)
    print("Total dates: {}".format(number_of_datetimes))

    number_of_test_dates=test_set_size*number_of_datetimes
    print("Test dates: {}".format(number_of_test_dates))

    test_index=int(number_of_datetimes - number_of_test_dates)
    print("Test index: {}".format(test_index))

    datetime_at_test_limit=sorted_datetimes[test_index]
    
    if verbose:
        print("Total dates: {}".format(number_of_datetimes))
        print("Test dates: {}".format(number_of_test_dates))
        print("Test index: {}".format(test_index))
        print("Dates after {} are test dates".format(datetime_at_test_limit))
        


    return datetime_at_test_limit

In [ ]:
def create_xgboost_model(fulldf,train_df,test_df,target_column,station_number,plot_comp,plot_tree):
    """Create an xgboostmodel"""
    
    X=fulldf.drop([target_column], axis=1)
    y=fulldf[target_column]
    
    X_test=test_df.drop([target_column], axis=1)
    y_test= test_df[target_column]
    
    X_train=train_df.drop([target_column], axis=1)
    y_train=train_df[target_column]
    
    
    #Paramter Dictionary
    
    model_parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7,8],
              'min_child_weight': [4],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

    #Create the XGBoost Regresspr
    xg_regression_model = xg.XGBRegressor(objective ='reg:squarederror')
    
    #Hypertune
    grid = GridSearchCV(xg_regression_model, model_parameters)
    grid.fit(X_train, y_train)
    
    best_parameters=grid.best_params_
    xg_regression_model = grid.best_estimator_


    #Score the model
    score=xg_regression_model.score(X_train,y_train)
    print("Model Training Score: {}%".format(score*100))
    
    #Check the predictions
    model_prediction = xg_regression_model.predict(X_test)
    
    kfold = KFold(n_splits=10)
    results = cross_val_score(xg_regression_model, X, y, cv=kfold)
    print("Model Accuracy: {}".format(results * 100))
    
    if plot_comp:
        #Original Versus Prediction
        print("The Original Vs Predicted Result Is:")
        plt.figure(figsize=(50,20)) 
        x_axis = range(len(y_test))
        plt.plot(x_axis, y_test, label="Original")
        plt.plot(x_axis, model_prediction, label="Predicted")
        plt.title("Station test and predicted data")
        plt.legend()
        plt.savefig('xg_pred_vs_orig_{}.png'.format(station_number))
        plt.show()
    
    
    filename = './xg_model_station_{}.pickle'
    
    xg_regression_model.save_model(filename.format(station_number))
    
    pred_vs_act_df=pd.DataFrame({'Actual':y_test,'Predicted':model_prediction})
    pred_vs_act_df['Predicted']=pred_vs_act_df['Predicted'].astype(int)
    pred_vs_act_df['Diff']=pred_vs_act_df['Actual']-pred_vs_act_df['Predicted']
    rmse=np.sqrt(mean_squared_error(y_test, model_prediction))
    print("RMSE: {}" .format(np.sqrt(mean_squared_error(y_test, model_prediction))))
    
    
    if plot_tree:
        #Visualisations, sometimes not great
        try:        

            #Tree Plot
            print("The Tree Is:")
            fig, ax = plt.subplots(figsize=(50, 20))
            xg.plot_tree(xg_regression_model,num_trees=2,ax=ax)
            plt.savefig('xg_tree_{}.png'.format(station_number))
            plt.show()

        
        
        
        except Exception as e:
            print(e)
        
    return [xg_regression_model,score,results,pred_vs_act_df,rmse]

In [ ]:
def create_linear_model(fulldf,train_df,test_df,target_column,station_number,plot_comp):
    """Create a linear model"""
    
    X=fulldf.drop([target_column], axis=1)
    y=fulldf[target_column]
    
    X_test=test_df.drop([target_column], axis=1)
    y_test= test_df[target_column]
    
    X_train=train_df.drop([target_column], axis=1)
    y_train=train_df[target_column]
    
    
    #Paramter Dictionary


    #Create the XGBoost Regresspr
    lin_regression_model = sk.linear_model.LinearRegression()


    #Fit the data
    lin_regression_model.fit(X_train,y_train)
    
    #Check the predictions
    lin_prediction = lin_regression_model.predict(X_test)
    
    if plot_comp:
        #Original Versus Prediction
        print("The Original Vs Predicted Result Is:")
        plt.figure(figsize=(50,20)) 
        x_axis = range(len(y_test))
        plt.plot(x_axis, y_test, label="Original")
        plt.plot(x_axis, lin_prediction, label="Predicted")
        plt.title("Station test and predicted data")
        plt.legend()
        plt.savefig('lin_pred_vs_orig_{}.png'.format(station_number))
        plt.show()
    
    filename = './lin_model_station_{}.pickle'
    pickle.dump(lin_regression_model, open(filename.format(station_number), 'wb'))
    
    pred_vs_act_df=pd.DataFrame({'Actual':y_test,'Predicted':lin_prediction})
    pred_vs_act_df['Predicted']=pred_vs_act_df['Predicted'].astype(int)
    pred_vs_act_df['Diff']=pred_vs_act_df['Actual']-pred_vs_act_df['Predicted']
    rmse=np.sqrt(mean_squared_error(y_test, lin_prediction))
    print("RMSE: {}" .format(np.sqrt(mean_squared_error(y_test,lin_prediction))))
        
    return [lin_regression_model,'','',pred_vs_act_df,rmse]

In [ ]:
#Get the raw data


myhost=database_dictionary['endpoint']
myuser=database_dictionary['username']
mypassword=database_dictionary['password']
myport=database_dictionary['port']
mydb=database_dictionary['database']



raw_df=station_availability_weather_table_df(host=myhost,user=myuser,password=mypassword,port=myport,db=mydb)  

In [ ]:
def generate_models(raw_df,host,user,password,port,db,plot_comp=True,plot_tree=True):
    """A function to generate models per station"""
        
        
    #HardCoded inputs - NOT A GOOD PRACTICE
    
    #Update time column
    update_time_column='entry_create_date'
    
    #Cleansed Columns
    cleansed_column_mapping={
                            'number': ['station_number']
                         , 'address': ['station_address']
                         , 'banking': ['station_banking']
                         , 'bike_status': ['station_bike_status']
                         , 'bike_stands': ['station_bike_stands']
                         ,  'contract_name': ['contract_name']
                         , 'name': ['station_name']
                         , 'position_lat': ['station_position_lat']
                         , 'position_long': ['station_position_lat']
                         ,  'available_bikes':['available_bikes']
                         , 'available_bike_stands':['available_bike_stands']
                         , 'last_update':['stand_last_update']
                         , 'created_date':[update_time_column]
                         , 'weather_position_long':['weather_position_long']
                         , 'weather_position_lat':['weather_position_lat']
                         ,  'weather_id':['weather_type_id']
                         , 'main':['weather_type_main']
                         , 'description':['weather_type_detail']
                         , 'icon':['weather_icon_type']
                         , 'icon_url':['weather_icon_url']
                         , 'base':['weather_base_name']
                         , 'temp':['weather_temp']
                         , 'feels_like':['weather_temp_feels_like']
                         , 'temp_min':['weather_temp_min']
                         , 'temp_max':['weather_temp_max']
                         , 'pressure':['weather_air_pressure']
                         , 'humidity':['weather_humidity']
                         , 'visibility':['weather_visibility']
                         , 'wind_speed':['weather_wind_speed']
                         , 'wind_degree':['weather_wind_direction']
                         , 'clouds_all':['clouds']
                         , 'datetime':['weather_datetime']
                         , 'sys_id':['weather_system_id']
                         , 'sys_country':['weather_country']
                         , 'sys_sunrise':['weather_sunrise_time']
                         , 'sys_sunset':['weather_sunrise_time']
                         , 'sys_type':['weather_station_type']
                         , 'timezone':['weather_timezone']
                         , 'id':['weather_pk']
                         , 'weather_name':['weather_place_name']
                         , 'cod':['weather_code']
                         }
    
    
    #Features - Initial
    relevant_feature_columns={
                    'feature':[
                                'station_number'
                               ,update_time_column
                               ,'weather_type_id'
                               ,'weather_temp'
                               ,'weather_temp_feels_like'
                               ,'weather_air_pressure'
                               ,'weather_humidity'
                              ]
    
                    ,'target':[
                                'available_bikes'
                              ]
                    
                            }
    
    
            
    engine_l=connect_db_engine(host,user,password,port,db)
    engine=engine_l[1]

    try:
        #Try Clear the DB
        
        enable_delete_SQL="""SET SQL_SAFE_UPDATES = 0"""
        disable_delete_SQL="""SET SQL_SAFE_UPDATES = 1"""

        engine.execute(enable_delete_SQL)
        
        engine.execute("""DELETE FROM 02_station_avail_weather_train WHERE 1=1""")
        engine.execute("""DELETE FROM 02_station_avail_weather_test WHERE 1=1""")
        
        engine.execute(disable_delete_SQL)
    except:
        print('Cannot Remove')

    #Staging Dataframe
    staging_df=raw_df.copy(deep=True)
    
    
    ###------
    #Rename Columns to Verbose - This should be functionised
    column_mapping={}
    
    for key,value in cleansed_column_mapping.items():
        for v in value:
            column_mapping[key] =  v

    
    staging_df=staging_df.rename(columns=column_mapping)
    
    #Column Renaming complete
    ###------
    
    
    
    ###------
    #Drop irrelevant features - This should be functionised
    keep_columns=[]

    for value in relevant_feature_columns.values():
        keep_columns+=value

    keep_columns=list(set(keep_columns))
    ###------
    
    
    ###------
    #Object column conversion to category for xgboost
    for object_column in ['station_number']:#,'weather_type_id']:
        
        #Check if in column list
        if object_column in staging_df.columns:
            
            #Change to category
            staging_df[object_column]=staging_df[object_column].astype('category')
            
        #Not in column list
        else:
            print("Missing {}".format(object_column))

    ###------
    staging_df=staging_df[keep_columns]
    
    #Add on day, hour, week
    time_feature_list=add_time_features(df=staging_df, date_time_column=update_time_column)
    
    #Staging Data with time features added
    staging_df=time_feature_list[0]
    
    #List of the time features added
    time_columns=time_feature_list[1]
    
    #Test Date Cutoff
    test_date=get_test_date(df=staging_df,time_column=update_time_column, test_set_size=0.2, verbose=False)

    #Get the average per hour - e.g. 5 10 stations, 1 0 station - avg is 50/6
    
    groupby_columns=[]
    
    for feature in relevant_feature_columns['feature']:
        if feature not in [update_time_column, 'station_number','weather_temp_feels_like','weather_temp','weather_humidity','weather_air_pressure']:
            groupby_columns+=[feature]
            
    for time_feature in time_columns:
        groupby_columns+=[time_feature]
    
    
    aggregation_dictionary={relevant_feature_columns['target'][0] : np.nanmean
                           ,'weather_temp_feels_like' : np.nanmean
                           ,'weather_temp' : np.nanmean
                           ,'weather_humidity' : np.nanmean
                           ,'weather_air_pressure' : np.nanmean
                           }
    
    print(time_columns)
    print(groupby_columns)
    print(aggregation_dictionary)
    
    
    station_dataframe_model_list={}
    

    
    
    datetime_now=dt.datetime.now()
    created_date=dt.datetime.timestamp(datetime_now)
    
    #For each station in the list
    for station_number in staging_df['station_number'].sort_values().unique():
        print("---------------")
        print("---------------")
        print("STATION {}".format(station_number))
        
        station_dataframe=pd.DataFrame()
        
            
        #Filter adf to that Dataframe
        station_dataframe=staging_df[staging_df['station_number']==station_number]
        
        #Station dataframe non-empty
        if len(station_dataframe)>0 and station_number!=507 and station_number!=508 and station_number!=509 and station_number!='507' and station_number!='508' and station_number!='509':
            
            #Drop feature of cardinality 1
            if 'station_number' in station_dataframe.columns:
                station_dataframe=station_dataframe.drop('station_number',axis=1)
                
            #Create Training and Test Split
            station_train_df=station_dataframe[station_dataframe[update_time_column] < test_date]
            station_test_df=station_dataframe[station_dataframe[update_time_column] >= test_date]
            
            
            station_dataframe=station_dataframe.drop(update_time_column,axis=1)
            station_train_df=station_train_df.drop(update_time_column,axis=1)
            station_test_df=station_test_df.drop(update_time_column,axis=1)
            
            #Need to do this here for filtering by date - will cause a slight imbalance in size via agg
            station_dataframe=(station_dataframe
                     .groupby(groupby_columns)
                     .agg(aggregation_dictionary)
                     .reset_index()
                    .dropna()
                )            
            
            #Need to do this here for filtering by date - will cause a slight imbalance in size via agg
            station_train_df=(station_train_df
                     .groupby(groupby_columns)
                     .agg(aggregation_dictionary)
                     .reset_index()
                    .dropna()
                )
            
            
            #Need to do this here for filtering by date - will cause a slight imbalance in size via agg
            station_test_df=(station_test_df
                     .groupby(groupby_columns)
                     .agg(aggregation_dictionary)
                     .reset_index()
                    .dropna()
                )
            
            
            
                
            #One hot encoding of Weather Type
            #station_dataframe=pd.get_dummies(station_dataframe, drop_first=True)
            #station_train_df=pd.get_dummies(station_train_df, drop_first=True)
            #station_test_df=pd.get_dummies(station_test_df, drop_first=True)
            
            try:
                #Don't touch the actual data
                temp_train_df=station_train_df.copy(deep=True)
                temp_test_df=station_test_df.copy(deep=True)

                #Add on created column
                temp_train_df['created_date']=created_date
                temp_test_df['created_date']=created_date

                #Station Number
                temp_train_df['number']=station_number
                temp_test_df['number']=station_number

                #Test/Train to DB
                temp_train_df.to_sql(name='02_station_avail_weather_train', con=engine, if_exists='append', index=False)
                temp_test_df.to_sql(name='02_station_avail_weather_test', con=engine, if_exists='append', index=False)
            
                #Remove these from memory
                del temp_train_df
                del temp_test_df

            except Exception as e:
                print("Exception posting testing and training data: {}".format(e))

            
            #Get station model
            model_result=create_xgboost_model(fulldf=station_dataframe,train_df=station_train_df,test_df=station_test_df,target_column='available_bikes',station_number=station_number,plot_comp=plot_comp,plot_tree=plot_tree)
            lin_model_result=create_linear_model(fulldf=station_dataframe,train_df=station_train_df,test_df=station_test_df,target_column='available_bikes',station_number=station_number,plot_comp=plot_comp)
                        
            #Get list of results to nester dict - Hefty RAM wise
            station_dataframe_model_list[station_number]={'model':model_result[0]
                                                         ,'score':model_result[1]
                                                         ,'results':model_result[2]
                                                         ,'predicted_vs_actual':model_result[3]
                                                         ,'rmse':model_result[4]
                                                        ,'lin_model':lin_model_result[0]
                                                         ,'lin_score':lin_model_result[1]
                                                         ,'lin_results':lin_model_result[2]
                                                         ,'lin_predicted_vs_actual':lin_model_result[3]
                                                         ,'lin_rmse':lin_model_result[4]
                                                         }
        
        #Pass Station
        else:
            print("No Data: {}".format(station_number))
            
    engine.dispose()
    
    return station_dataframe_model_list

In [ ]:

database_schema={
                    '01_station':{
                              'address':'VARCHAR(256)'
                              ,'banking':'INTEGER'
                              ,'bike_stands':'INTEGER'
                              ,'bonus':'INTEGER'
                              ,'contract_name':'VARCHAR(256)'
                              ,'name':'VARCHAR(256)'
                              ,'number':'INTEGER'
                              ,'position_lat':'REAL'
                              ,'position_long':'REAL'
                              ,'created_date': 'BIGINT'
                                }
                    ,'01_availability':{
                                'number':'INTEGER'
                                ,'available_bikes':'INTEGER'
                                ,'available_bike_stands':'INTEGER'
                                ,'last_update':'BIGINT'
                                ,'created_date':'BIGINT'
                                }
    
                    ,'01_weather':{
                                'number': 'INT'
                                ,'position_long':'REAL'
                                ,'position_lat':'REAL'
                                ,'weather_id':'INTEGER'
                                ,'main':'VARCHAR(256)'
                                ,'description':'VARCHAR(500)'
                                ,'icon':'VARCHAR(20)'
                                ,'icon_url':'VARCHAR(500)'
                                ,'base':'varchar(256)'
                                ,'temp':'REAL'
                                ,'feels_like':'REAL'
                                ,'temp_min':'REAL'
                                ,'temp_max':'REAL'
                                ,'pressure':'INT'
                                ,'humidity':'INT'
                                ,'visibility':'INT'
                                ,'wind_speed':'REAL'
                                ,'wind_degree':'INT'
                                ,'clouds_all':'INT'
                                ,'datetime':'BIGINT'
                                ,'sys_id':'INT'
                                ,'sys_country':'VARCHAR(10)'
                                ,'sys_sunrise':'BIGINT'
                                ,'sys_sunset':'BIGINT'
                                ,'sys_type':'INT'
                                ,'timezone':'INT'
                                ,'id':'BIGINT'
                                ,'name':'VARCHAR(256)'
                                ,'cod':'INT'
                                ,'created_date':'BIGINT'
                                }

                    ,'01_forecast':{
        # Removed base, timezone, avail_update_dt, datetime, id_var, name, cod and all 'sys' entries when compared to original scraper
        # Added forecast_time_dt and forecast_time_txt to show the forecast times in readable format
                                'number': 'INT'
                                ,'position_long':'REAL'
                                ,'position_lat':'REAL'
                                ,'weather_id':'INTEGER'
                                ,'main':'VARCHAR(256)'
                                ,'description':'VARCHAR(500)'
                                ,'icon':'VARCHAR(20)'
                                ,'icon_url':'VARCHAR(500)'
                                ,'temp':'REAL'
                                ,'feels_like':'REAL'
                                ,'temp_min':'REAL'
                                ,'temp_max':'REAL'
                                ,'pressure':'INT'
                                ,'humidity':'INT'
                                ,'visibility':'INT'
                                ,'wind_speed':'REAL'
                                ,'wind_degree':'INT'
                                ,'clouds_all':'INT'
                                ,'forecast_time_ts':'BIGINT'
                                ,'forecast_time_dt':'DATETIME'
                                ,'created_date':'BIGINT'
                                }
    
                    ,'02_station_avail_weather_train':{
                                'number':'INTEGER'
                                , 'weather_type_id':'INTEGER'
                                , 'hour':'INTEGER'
                                , 'dayofweek':'INTEGER'
                                , 'dayofmonth':'INTEGER'
                                , 'bool_weekend':'BOOLEAN'
                                , 'bool_dayoff':'BOOLEAN'
                                , 'bool_workhour':'BOOLEAN'
                                , 'bool_commutehour':'BOOLEAN'
                                , 'bool_night':'BOOLEAN'
                                , 'available_bikes':'REAL' #average over everything
                                , 'weather_temp_feels_like':'REAL' #average over everything
                                , 'weather_temp':'REAL' #average over everything
                               ,  'weather_humidity':'REAL' #average over everything
                                , 'weather_air_pressure':'REAL' #average over everything
                                , 'created_date':'BIGINT'
                                ,'minute':'INTEGER'
                                ,'month':'INTEGER'
                                ,'year':'INTEGER'
                                }
    
,                   '02_station_avail_weather_test':{
                                'number':'INTEGER'
                                , 'weather_type_id':'INTEGER'
                                , 'hour':'INTEGER'
                                , 'dayofweek':'INTEGER'
                                , 'dayofmonth':'INTEGER'
                                , 'bool_weekend':'BOOLEAN'
                                , 'bool_dayoff':'BOOLEAN'
                                , 'bool_workhour':'BOOLEAN'
                                , 'bool_commutehour':'BOOLEAN'
                                , 'bool_night':'BOOLEAN'
                                , 'available_bikes':'REAL' #average over everything
                                , 'weather_temp_feels_like':'REAL' #average over everything
                                , 'weather_temp':'REAL' #average over everything
                                , 'weather_humidity':'REAL' #average over everything
                                , 'weather_air_pressure':'REAL' #average over everything
                                , 'created_date':'BIGINT'
                                ,'minute':'INTEGER'
                                ,'month':'INTEGER'
                                ,'year':'INTEGER'
                                }
    
    ,                   '03_user_model_entry':{
                                'prediction_id':'INTEGER NOT NULL AUTO_INCREMENT'
                                ,'number':'INTEGER'
                                , 'weather_type_id':'INTEGER'
                                , 'hour':'INTEGER'
                                , 'dayofweek':'INTEGER'
                                , 'dayofmonth':'INTEGER'
                                , 'bool_weekend':'BOOLEAN'
                                , 'bool_dayoff':'BOOLEAN'
                                , 'bool_workhour':'BOOLEAN'
                                , 'bool_commutehour':'BOOLEAN'
                                , 'bool_night':'BOOLEAN'
                                , 'available_bikes':'REAL' #average over everything
                                , 'weather_temp_feels_like':'REAL' #average over everything
                                , 'weather_temp':'REAL' #average over everything
                                , 'weather_humidity':'REAL' #average over everything
                                , 'weather_air_pressure':'REAL' #average over everything
                                , 'user_date':'BIGINT'        
                                , 'created_date':'BIGINT'
                                , 'predicted_date':'BIGINT'
                                , 'model_type':'VARCHAR(100)'
                                , 'bool_correct_prediction':'BOOLEAN'
                                ,'minute':'INTEGER'
                                ,'month':'INTEGER'
                                ,'year':'INTEGER'
                                }
                }



In [ ]:
model_data=generate_models(raw_df=raw_df,host=myhost,user=myuser,password=mypassword,port=myport,db=mydb,plot_comp=True,plot_tree=True)

In [ ]:
raw_df.describe()

In [ ]:
model_data[2]['predicted_vs_actual']['Actual']=model_data[2]['predicted_vs_actual']['Actual'].astype(int)

In [ ]:
model_data[2]['predicted_vs_actual']['Diff']=model_data[2]['predicted_vs_actual']['Actual'] - model_data[2]['predicted_vs_actual']['Predicted']

In [ ]:
model_data[4]['lin_predicted_vs_actual'].mean()

In [ ]:

model_data[4]['predicted_vs_actual'].mean()

In [ ]:
xgbu=0
xgbl=0
xgbb=0
lin_win=[]
xg_win=[]
for a in model_data.keys():
    if model_data[a]['rmse']<=model_data[a]['lin_rmse']:
        xg_win+=[a]
        xgbb+=model_data[a]['rmse']
    else:
        lin_win+=[a]
        xgbl+=model_data[a]['lin_rmse']

In [ ]:
xg_win

In [ ]:
lin_win

In [ ]:

database_schema={
                    '01_station':{
                              'address':'VARCHAR(256)'
                              ,'banking':'INTEGER'
                              ,'bike_stands':'INTEGER'
                              ,'bonus':'INTEGER'
                              ,'contract_name':'VARCHAR(256)'
                              ,'name':'VARCHAR(256)'
                              ,'number':'INTEGER'
                              ,'position_lat':'REAL'
                              ,'position_long':'REAL'
                              ,'created_date': 'BIGINT'
                                }
                    ,'01_availability':{
                                'number':'INTEGER'
                                ,'available_bikes':'INTEGER'
                                ,'available_bike_stands':'INTEGER'
                                ,'last_update':'BIGINT'
                                ,'created_date':'BIGINT'
                                }

                    ,'02_station_avail_weather_train':{
                                'number':'INTEGER'
                                ,'available_bikes':'INTEGER'
                                ,'available_bike_stands':'INTEGER'
                                ,'last_update':'BIGINT'
                                ,'created_date':'BIGINT'
                                }
,                   '02_station_avail_weather_test':{
                                'number':'INTEGER'
                                ,'available_bikes':'INTEGER'
                                ,'available_bike_stands':'INTEGER'
                                ,'last_update':'BIGINT'
                                ,'created_date':'BIGINT'
                                }
                    ,'01_weather':{
                                'number': 'INT'
                                ,'position_long':'REAL'
                                ,'position_lat':'REAL'
                                ,'weather_id':'INTEGER'
                                ,'main':'VARCHAR(256)'
                                ,'description':'VARCHAR(500)'
                                ,'icon':'VARCHAR(20)'
                                ,'icon_url':'VARCHAR(500)'
                                ,'base':'varchar(256)'
                                ,'temp':'REAL'
                                ,'feels_like':'REAL'
                                ,'temp_min':'REAL'
                                ,'temp_max':'REAL'
                                ,'pressure':'INT'
                                ,'humidity':'INT'
                                ,'visibility':'INT'
                                ,'wind_speed':'REAL'
                                ,'wind_degree':'INT'
                                ,'clouds_all':'INT'
                                ,'datetime':'BIGINT'
                                ,'sys_id':'INT'
                                ,'sys_country':'VARCHAR(10)'
                                ,'sys_sunrise':'BIGINT'
                                ,'sys_sunset':'BIGINT'
                                ,'sys_type':'INT'
                                ,'timezone':'INT'
                                ,'id':'BIGINT'
                                ,'name':'VARCHAR(256)'
                                ,'cod':'INT'
                                ,'created_date':'BIGINT'
                                }

                    ,'01_forecast':{
        # Removed base, timezone, avail_update_dt, datetime, id_var, name, cod and all 'sys' entries when compared to original scraper
        # Added forecast_time_dt and forecast_time_txt to show the forecast times in readable format
                                'number': 'INT'
                                ,'position_long':'REAL'
                                ,'position_lat':'REAL'
                                ,'weather_id':'INTEGER'
                                ,'main':'VARCHAR(256)'
                                ,'description':'VARCHAR(500)'
                                ,'icon':'VARCHAR(20)'
                                ,'icon_url':'VARCHAR(500)'
                                ,'temp':'REAL'
                                ,'feels_like':'REAL'
                                ,'temp_min':'REAL'
                                ,'temp_max':'REAL'
                                ,'pressure':'INT'
                                ,'humidity':'INT'
                                ,'visibility':'INT'
                                ,'wind_speed':'REAL'
                                ,'wind_degree':'INT'
                                ,'clouds_all':'INT'
                                ,'forecast_time_dt':'BIGINT'
                                ,'forecast_time_txt':'DATETIME'
                                ,'created_date':'BIGINT'
                                }
                }


In [ ]:
    data={
'weather_air_pressure'       :  [1029],
'weather_temp_feels_like'   : [276.64],
'weather_type_id'          :    [803],
'weather_humidity'        :     [83],
'weather_temp'           :    [278.84],
'hour'                  :       [8],
'dayofweek'            :      [6],
'dayofmonth'          :         [10],
'bool_weekend'       :           [True],
'bool_dayoff'       :            [True],
'bool_workhour'    :             [True],
'bool_commutehour':              [False],
'bool_night':                    [False]
}

prdf=pd.DataFrame(data)

In [ ]:
prdf=prdf[['weather_type_id',
 'hour',
 'dayofweek',
 'dayofmonth',
 'bool_weekend',
 'bool_dayoff',
 'bool_workhour',
 'bool_commutehour',
 'bool_night',
 'weather_temp_feels_like',
 'weather_temp',
 'weather_humidity',
 'weather_air_pressure']]

In [ ]:
model_data[2]['lin_model'].predict(prdf)#.get_booster().feature_names

In [ ]:
prdf.dtypes

In [ ]:
def continue_generate_models(raw_df,host,user,password,port,db,plot_comp=True,plot_tree=True):
    """A function to generate models per station"""
        
        
    #HardCoded inputs - NOT A GOOD PRACTICE
    
    #Update time column
    update_time_column='entry_create_date'
    
    #Cleansed Columns
    cleansed_column_mapping={
                            'number': ['station_number']
                         , 'address': ['station_address']
                         , 'banking': ['station_banking']
                         , 'bike_status': ['station_bike_status']
                         , 'bike_stands': ['station_bike_stands']
                         ,  'contract_name': ['contract_name']
                         , 'name': ['station_name']
                         , 'position_lat': ['station_position_lat']
                         , 'position_long': ['station_position_lat']
                         ,  'available_bikes':['available_bikes']
                         , 'available_bike_stands':['available_bike_stands']
                         , 'last_update':['stand_last_update']
                         , 'created_date':[update_time_column]
                         , 'weather_position_long':['weather_position_long']
                         , 'weather_position_lat':['weather_position_lat']
                         ,  'weather_id':['weather_type_id']
                         , 'main':['weather_type_main']
                         , 'description':['weather_type_detail']
                         , 'icon':['weather_icon_type']
                         , 'icon_url':['weather_icon_url']
                         , 'base':['weather_base_name']
                         , 'temp':['weather_temp']
                         , 'feels_like':['weather_temp_feels_like']
                         , 'temp_min':['weather_temp_min']
                         , 'temp_max':['weather_temp_max']
                         , 'pressure':['weather_air_pressure']
                         , 'humidity':['weather_humidity']
                         , 'visibility':['weather_visibility']
                         , 'wind_speed':['weather_wind_speed']
                         , 'wind_degree':['weather_wind_direction']
                         , 'clouds_all':['clouds']
                         , 'datetime':['weather_datetime']
                         , 'sys_id':['weather_system_id']
                         , 'sys_country':['weather_country']
                         , 'sys_sunrise':['weather_sunrise_time']
                         , 'sys_sunset':['weather_sunrise_time']
                         , 'sys_type':['weather_station_type']
                         , 'timezone':['weather_timezone']
                         , 'id':['weather_pk']
                         , 'weather_name':['weather_place_name']
                         , 'cod':['weather_code']
                         }
    
    
    #Features - Initial
    relevant_feature_columns={
                    'feature':[
                                'station_number'
                               ,update_time_column
                               ,'weather_type_id'
                               ,'weather_temp'
                               ,'weather_temp_feels_like'
                               ,'weather_air_pressure'
                               ,'weather_humidity'
                              ]
    
                    ,'target':[
                                'available_bikes'
                              ]
                    
                            }
    
    
            
    engine_l=connect_db_engine(host,user,password,port,db)
    engine=engine_l[1]


    #Staging Dataframe
    staging_df=raw_df.copy(deep=True)
    
    
    ###------
    #Rename Columns to Verbose - This should be functionised
    column_mapping={}
    
    for key,value in cleansed_column_mapping.items():
        for v in value:
            column_mapping[key] =  v

    
    staging_df=staging_df.rename(columns=column_mapping)
    
    #Column Renaming complete
    ###------
    
    
    
    ###------
    #Drop irrelevant features - This should be functionised
    keep_columns=[]

    for value in relevant_feature_columns.values():
        keep_columns+=value

    keep_columns=list(set(keep_columns))
    ###------
    
    
    ###------
    #Object column conversion to category for xgboost
    for object_column in ['station_number']:#,'weather_type_id']:
        
        #Check if in column list
        if object_column in staging_df.columns:
            
            #Change to category
            staging_df[object_column]=staging_df[object_column].astype('category')
            
        #Not in column list
        else:
            print("Missing {}".format(object_column))

    ###------
    staging_df=staging_df[keep_columns]
    
    #Add on day, hour, week
    time_feature_list=add_time_features(df=staging_df, date_time_column=update_time_column)
    
    #Staging Data with time features added
    staging_df=time_feature_list[0]
    
    #List of the time features added
    time_columns=time_feature_list[1]
    
    #Test Date Cutoff
    test_date=get_test_date(df=staging_df,time_column=update_time_column, test_set_size=0.2, verbose=False)

    #Get the average per hour - e.g. 5 10 stations, 1 0 station - avg is 50/6
    
    groupby_columns=[]
    
    for feature in relevant_feature_columns['feature']:
        if feature not in [update_time_column, 'station_number','weather_temp_feels_like','weather_temp','weather_humidity','weather_air_pressure']:
            groupby_columns+=[feature]
            
    for time_feature in time_columns:
        groupby_columns+=[time_feature]
    
    
    aggregation_dictionary={relevant_feature_columns['target'][0] : np.nanmean
                           ,'weather_temp_feels_like' : np.nanmean
                           ,'weather_temp' : np.nanmean
                           ,'weather_humidity' : np.nanmean
                           ,'weather_air_pressure' : np.nanmean
                           }
    
    print(time_columns)
    print(groupby_columns)
    print(aggregation_dictionary)
    
    
    station_dataframe_model_list={}
    

    
    
    datetime_now=dt.datetime.now()
    created_date=dt.datetime.timestamp(datetime_now)
    
    #For each station in the list
    for station_number in staging_df['station_number'].sort_values().unique():
        print("---------------")
        print("---------------")
        print("STATION {}".format(station_number))
        
        station_dataframe=pd.DataFrame()
        
            
        #Filter adf to that Dataframe
        station_dataframe=staging_df[staging_df['station_number']==station_number]
        
        #Station dataframe non-empty
        if len(station_dataframe)>0 and int(station_number)==114:
            
            #Drop feature of cardinality 1
            if 'station_number' in station_dataframe.columns:
                station_dataframe=station_dataframe.drop('station_number',axis=1)
                
            #Create Training and Test Split
            station_train_df=station_dataframe[station_dataframe[update_time_column] < test_date]
            station_test_df=station_dataframe[station_dataframe[update_time_column] >= test_date]
            
            
            station_dataframe=station_dataframe.drop(update_time_column,axis=1)
            station_train_df=station_train_df.drop(update_time_column,axis=1)
            station_test_df=station_test_df.drop(update_time_column,axis=1)
            
            #Need to do this here for filtering by date - will cause a slight imbalance in size via agg
            station_dataframe=(station_dataframe
                     .groupby(groupby_columns)
                     .agg(aggregation_dictionary)
                     .reset_index()
                    .dropna()
                )            
            
            #Need to do this here for filtering by date - will cause a slight imbalance in size via agg
            station_train_df=(station_train_df
                     .groupby(groupby_columns)
                     .agg(aggregation_dictionary)
                     .reset_index()
                    .dropna()
                )
            
            
            #Need to do this here for filtering by date - will cause a slight imbalance in size via agg
            station_test_df=(station_test_df
                     .groupby(groupby_columns)
                     .agg(aggregation_dictionary)
                     .reset_index()
                    .dropna()
                )
            
            
            
                
            #One hot encoding of Weather Type
            #station_dataframe=pd.get_dummies(station_dataframe, drop_first=True)
            #station_train_df=pd.get_dummies(station_train_df, drop_first=True)
            #station_test_df=pd.get_dummies(station_test_df, drop_first=True)
            
            try:
                #Don't touch the actual data
                temp_train_df=station_train_df.copy(deep=True)
                temp_test_df=station_test_df.copy(deep=True)

                #Add on created column
                temp_train_df['created_date']=created_date
                temp_test_df['created_date']=created_date

                #Station Number
                temp_train_df['number']=station_number
                temp_test_df['number']=station_number

                #Test/Train to DB
                temp_train_df.to_sql(name='02_station_avail_weather_train', con=engine, if_exists='append', index=False)
                temp_test_df.to_sql(name='02_station_avail_weather_test', con=engine, if_exists='append', index=False)
            
                #Remove these from memory
                del temp_train_df
                del temp_test_df

            except Exception as e:
                print("Exception posting testing and training data: {}".format(e))

            
            #Get station model
            model_result=create_xgboost_model(fulldf=station_dataframe,train_df=station_train_df,test_df=station_test_df,target_column='available_bikes',station_number=station_number,plot_comp=plot_comp,plot_tree=plot_tree)
            lin_model_result=create_linear_model(fulldf=station_dataframe,train_df=station_train_df,test_df=station_test_df,target_column='available_bikes',station_number=station_number,plot_comp=plot_comp)
                        
            #Get list of results to nester dict - Hefty RAM wise
            station_dataframe_model_list[station_number]={'model':model_result[0]
                                                         ,'score':model_result[1]
                                                         ,'results':model_result[2]
                                                         ,'predicted_vs_actual':model_result[3]
                                                         ,'rmse':model_result[4]
                                                        ,'lin_model':lin_model_result[0]
                                                         ,'lin_score':lin_model_result[1]
                                                         ,'lin_results':lin_model_result[2]
                                                         ,'lin_predicted_vs_actual':lin_model_result[3]
                                                         ,'lin_rmse':lin_model_result[4]
                                                         }
        
        #Pass Station
        else:
            print("No Data: {}".format(station_number))
            
    engine.dispose()
    
    return station_dataframe_model_list

In [ ]:
model_data=continue_generate_models(raw_df=raw_df,host=myhost,user=myuser,password=mypassword,port=myport,db=mydb,plot_comp=True,plot_tree=True)